In [1]:
# import necessary dependencies
import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm

import os
import torch
import torch.nn as nn
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torchvision

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
import random
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

In [3]:
BATCH_SIZE = 128
EPOCHS = 200
LR = 1e-4

In [4]:
# train_transform = transforms.Compose(
#     [transforms.Resize(size=(32, 32)),
#     transforms.ToTensor()]
# )

color_jitter = torchvision.transforms.ColorJitter(
        0.4, 0.4, 0.4, 0.1
    )
train_transform = torchvision.transforms.Compose(
    [
        transforms.RandomResizedCrop(size=(32, 32)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomApply([color_jitter], p=0.8),
        torchvision.transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
    ]
)

test_transform = transforms.Compose(
    [transforms.Resize(size=(32, 32)),
    transforms.ToTensor()]
)

In [5]:
trainset = torchvision.datasets.CIFAR10(root='/usr/xtmp/zg78/cifar10_data/', train=True, download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='/usr/xtmp/zg78/cifar10_data/', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
criterion = nn.CrossEntropyLoss()

In [ ]:
net = torchvision.models.resnet18().cuda()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

best_loss = 9999999

for epoch_idx in range(EPOCHS):
    epoch_losses = 0
    epoch_correts = 0
    net.train()
    for batch_idx, data in enumerate(tqdm(trainloader)):
        image, label = data
        image = image.cuda()
        label = label.cuda()
        
        net.zero_grad()
        out = net(image)
        
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()

        epoch_losses += loss
    
        pred = torch.argmax(out, dim=1)
        epoch_correts += torch.sum(pred == label).item()
    
    epoch_losses /= len(trainloader)
    epoch_correts /= len(trainset)
        
    
    with torch.no_grad():
        net.eval()
        
        val_epoch_losses = 0
        val_epoch_correts = 0

        for batch_idx, data in enumerate(tqdm(testloader)):
            image, label = data
            image = image.cuda()
            label = label.cuda()

            out = net(image)

            loss = criterion(out, label)

            val_epoch_losses += loss

            pred = torch.argmax(out, dim=1)
            val_epoch_correts += torch.sum(pred == label).item()

        val_epoch_losses /= len(testloader)
        val_epoch_correts /= len(testset)

        if val_epoch_losses < best_loss:
            best_loss = val_epoch_losses
            torch.save(net.state_dict(), f'resent18_baseline_{EPOCHS}_{BATCH_SIZE}_{LR}.pth')
    
    print(f'Train Loss {epoch_losses} Acc {epoch_correts} ; Val Loss {val_epoch_losses} Acc {val_epoch_correts}')

  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

Train Loss 2.0941312313079834 Acc 0.2606 ; Val Loss 1.6429963111877441 Acc 0.396


  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

...

Train Loss 0.6989233493804932 Acc 0.75544 ; Val Loss 0.5463509559631348 Acc 0.8237


  0%|          | 0/391 [00:00<?, ?it/s]

  0%|          | 0/79 [00:00<?, ?it/s]

Train Loss 0.6872727274894714 Acc 0.75926 ; Val Loss 0.5410946011543274 Acc 0.8327


  0%|          | 0/391 [00:00<?, ?it/s]